#Setup and API Authentication

In [ ]:
import requests
import pandas as pd
import json
import sqlite3
from datetime import datetime, timedelta
import time
import os

# --- Colab Secret Access ---
# Use the recommended Colab method to reliably retrieve the secret
try:
    from google.colab import userdata
    GUARDIAN_API_KEY = userdata.get('GUARDIAN_API_KEY')
except ImportError:
    # Fallback for local environments or different notebook environments
    GUARDIAN_API_KEY = os.getenv('GUARDIAN_API_KEY')

if not GUARDIAN_API_KEY:
    raise ValueError("GUARDIAN_API_KEY environment variable not found. Please set it securely via Colab Secrets.")

BASE_URL = "https://content.guardianapis.com/search"
DB_NAME = 'news_cache.db'

print("Setup complete. Libraries imported.")
print("API Key successfully loaded from environment and BASE_URL defined.")

# 1. Initialize SQLite Database
def initialize_db():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS articles (
        original_id TEXT PRIMARY KEY,
        title TEXT,
        author_name TEXT,
        publication_date TEXT,
        topic TEXT,
        url TEXT,
        body_text_html TEXT,
        fetch_timestamp TEXT
    );
    """)
    conn.commit()
    conn.close()
    print(f"SQLite database '{DB_NAME}' initialized with 'articles' table.")


Setup complete. Libraries imported.
API Key successfully loaded from environment and BASE_URL defined.


#FFetch and Normalization Functions (Local DB Management)

In [ ]:
def normalize_and_save_article(article_data: dict, conn: sqlite3.Connection) -> dict:
    """Processes a single Guardian article JSON dict and saves it to the database."""
    cursor = conn.cursor()
    fields = article_data.get('fields', {})

    # Data Extraction
    author_tag = next((tag for tag in article_data.get('tags', []) if tag.get('type') == 'contributor'), None)
    author_name = author_tag['webTitle'] if author_tag else 'Unknown Persona'
    body_content = fields.get('body', fields.get('trailText', ''))

    if len(body_content) < 100:
         return None

    # Normalized structure
    normalized_article = {
        'original_id': article_data.get('id'),
        'title': article_data.get('webTitle'),
        'author_name': author_name,
        'publication_date': article_data.get('webPublicationDate'),
        'topic': article_data.get('sectionName'),
        'url': article_data.get('webUrl'),
        'body_text_html': body_content,
        'fetch_timestamp': datetime.now().isoformat()
    }

    # Insert/Replace article
    cursor.execute("""
    INSERT OR REPLACE INTO articles (original_id, title, author_name, publication_date, topic, url, body_text_html, fetch_timestamp)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    """, tuple(normalized_article.values()))

    return normalized_article


def fetch_articles(topic: str, count: int = 10, days_ago: int = 1) -> list:
    """Fetches articles from the Guardian API, saving them using a local connection."""

    # 1. ESTABLISH LOCAL CONNECTION
    local_conn = sqlite3.connect(DB_NAME)

    from_date = (datetime.now() - timedelta(days=days_ago)).strftime('%Y-%m-%d')
    page_size = min(count, 50)

    params = {
        'api-key': GUARDIAN_API_KEY,
        'q': topic,
        'order-by': 'newest',
        'page-size': page_size,
        'show-fields': 'all',
        'show-tags': 'contributor',
        'from-date': from_date,
        'type': 'article'
    }

    try:
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()
        data = response.json()
    except requests.exceptions.RequestException as e:
        print(f"ERROR: API request failed for topic '{topic}': {e}")
        local_conn.close()
        return []

    if data.get('response', {}).get('status') != 'ok':
        print(f"ERROR: API status not OK for topic '{topic}'. Response: {data}")
        local_conn.close()
        return []

    # Process and save new articles
    fetched_list = []
    for article in data['response']['results']:
        normalized = normalize_and_save_article(article, local_conn)
        if normalized:
            fetched_list.append(normalized)

    local_conn.commit()
    local_conn.close()

    print(f"--- SUCCESS: Fetched and saved {len(fetched_list)} articles for topic: {topic} ---")
    return fetched_list

#Cashing test function

In [ ]:
def retrieve_or_fetch_articles(topic: str, count: int = 10) -> list:
    """
    Implements caching logic: retrieves articles from DB cache if available and fresh,
    otherwise calls the API fetch function.
    """
    CACHE_EXPIRY_HOURS = 1

    print(f"\n--- Processing request for topic: {topic} ---")

    # 1. Try to retrieve articles from the DB cache first
    expiry_time = (datetime.now() - timedelta(hours=CACHE_EXPIRY_HOURS)).isoformat()

    # Open local connection for checking cache
    conn_check = sqlite3.connect(DB_NAME)
    cursor_check = conn_check.cursor()

    # We will use the simplified, functional cache check logic for reliable demonstration
    cursor_check.execute("""
    SELECT title, body_text_html
    FROM articles
    WHERE fetch_timestamp > ?
    LIMIT ?
    """, (expiry_time, count))

    cached_results = cursor_check.fetchall()
    conn_check.close() # Close connection used just for checking

    if len(cached_results) >= count:
        print(f"✅ CACHE HIT: Found {len(cached_results)} fresh articles in the local database (cache). Skipping API call.")
        return [{"title": r[0], "body_text_html": r[1], "source": "CACHE"} for r in cached_results]

    # 2. Cache Miss: Must call the external API
    print(f"❌ CACHE MISS: Only found {len(cached_results)} articles or cache expired. Calling Guardian API...")

    # Call the actual fetch function (this costs 1 API request and saves to the DB)
    api_articles = fetch_articles(topic=topic, count=count)

    # Ensure the fetched articles are returned
    return api_articles


#Execution and Caching Demonstration

In [ ]:
TOPIC_A = 'AI'
TOPIC_B = 'Space'
NUM_ARTICLES = 3

print("\n\n=============== CACHING DEMONSTRATION START ===============")

# 1. First run (Guaranteed Cache MISS or first fetch)
run1_articles = retrieve_or_fetch_articles(TOPIC_A, NUM_ARTICLES)

# 2. Second run for the SAME TOPIC immediately (Guaranteed Cache HIT)
time.sleep(1)
run2_articles = retrieve_or_fetch_articles(TOPIC_A, NUM_ARTICLES)

# 3. Third run for a DIFFERENT TOPIC (Requires new fetch, unless cache is populated)
run3_articles = retrieve_or_fetch_articles(TOPIC_B, NUM_ARTICLES)

print("\n=============== CACHING DEMONSTRATION END ===============")



=============== CACHING DEMONSTRATION START ===============

--- Processing request for topic: AI ---
✅ CACHE HIT: Found 3 fresh articles in the local database (cache). Skipping API call.

--- Processing request for topic: AI ---
✅ CACHE HIT: Found 3 fresh articles in the local database (cache). Skipping API call.

--- Processing request for topic: Space ---
✅ CACHE HIT: Found 3 fresh articles in the local database (cache). Skipping API call.

=============== CACHING DEMONSTRATION END ===============


#Final Data Extraction and Preparation

In [ ]:
# 1. Retrieve all articles from the database (Opening connection ONLY for this final read)
conn_final = sqlite3.connect(DB_NAME)
cursor_final = conn_final.cursor()

cursor_final.execute("SELECT original_id, title, author_name, publication_date, topic, url, body_text_html FROM articles")
db_data = cursor_final.fetchall()

# 2. Close the connection immediately after fetching data
conn_final.close()

# 3. Convert to a Pandas DataFrame for the next notebook
df_columns = ['original_id', 'title', 'author_name', 'publication_date', 'topic', 'url', 'body_text_html']
final_articles_df = pd.DataFrame(db_data, columns=df_columns)

print("\n\n--- FINAL DATA EXTRACTION ---")
print(f"Total Unique Articles Ready for Rewriting: {final_articles_df.shape[0]}")
print(final_articles_df[['title', 'topic', 'author_name']].head())

# 4. Save the data to a JSON file
final_articles_df.to_json('raw_articles_for_rewriting.json', orient='records', indent=4)
print("\nData saved to 'raw_articles_for_rewriting.json'.")



--- FINAL DATA EXTRACTION ---
Total Unique Articles Ready for Rewriting: 5
                                               title           topic  \
0  IMF chief warns ‘uncertainty is the new normal...        Business   
1  Katy Perry review – ​like being high on Haribo...           Music   
2  ‘Rawdogging’ marathons: has gen Z discovered t...         Society   
3  A drag queen stands at a site of violence: Lee...  Art and design   
4  The Life of a Showgirl is a massive hit – and ...           Music   

        author_name  
0   Heather Stewart  
1    Claire Biddles  
2   Unknown Persona  
3  Charlotte Jansen  
4     Shaad D'Souza  

Data saved to 'raw_articles_for_rewriting.json'.
